In [1]:
# Process starts: Debug training setup.
import sys
import os
import torch
from torch.utils.data import DataLoader
# Add project root to path.
project_root = '/root/autodl-tmp/Adaptive-Hybrid-Framework-For-Enhanced-3D-Object-Classification'
sys.path.insert(0, project_root)
from datasets.modelnet_dataset import ModelNetDataset
from training.train import get_model, train_one_epoch
from configs.modelnet40_config import dataset, model as model_config
from core.dynamic_voxelizer import DynamicVoxelizer

try:
    # Setup device.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Load dataset.
    train_dataset = ModelNetDataset(root=dataset['root'], train=True, num_points=dataset['num_points'])
    train_loader = DataLoader(train_dataset, batch_size=dataset['batch_size'], shuffle=True)
    print(f"Loaded {len(train_dataset)} training samples.")

    # Load model and voxelizer.
    model = get_model(model_config).to(device)
    voxelizer = DynamicVoxelizer(voxel_size=0.05, max_voxels=100000).to(device)
    print("Model and voxelizer initialized successfully.")

    # Test one batch.
    points, labels = next(iter(train_loader))
    points, labels = points.to(device), labels.to(device)
    print(f"Batch shape: {points.shape}, Labels shape: {labels.shape}")  # Expected: [32, 2048, 3], [32]
    
    # Test voxelizer.
    voxels = voxelizer(points)
    print(f"Voxelized tensor: {voxels.features.shape}, {voxels.indices.shape}")

    # Test forward pass.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss, acc = train_one_epoch(model, train_loader, optimizer, device, epoch=1)
    print(f"Test forward pass loss: {loss:.4f}, accuracy: {acc:.4f}")

except Exception as e:
    print(f"Error in training setup: {e}")
    raise
# Process ends: Debug complete.

ModuleNotFoundError: No module named 'configs.modelnet40_config'